In [7]:
def extract_MFCC(filepath):
    audio, sample_rate = librosa.load(filepath , res_type="kaiser_fast")
    mfccs=np.mean(librosa.feature.mfcc(y=audio, sr=sample_rate , n_mfcc=100).T,axis=0)
    return mfccs

In [27]:
from scipy import spatial
def calculate_accuracy(x_test, y_test, model):
    predictions=model.predict(x_test)
    predictions /= np.linalg.norm(predictions, axis=1, keepdims=True)
    div=0.8 #total classes that are predicted
    per_acc=[]
    for i in range(0,x_test.shape[0], 10):
        a=predictions[i]
        label_1= np.argmax(y_test[i])
        test_acc=[]
        print("Calculating the {} th sample".format(i//10))
        for j in range(x_test.shape[0]):
            b=predictions[j]
            if np.mean(a) == np.mean(b):
                continue
            label_2 = np.argmax(y_test[j])
            sim = 1- spatial.distance.cosine(a, b)
            test_acc.append((sim, label_1, label_2))
            
        sorted_test = sorted(test_acc, reverse=True)
        print(sorted_test[:9])
        per_acc.append(sorted_test[:9])
    
    total_acc=[]
    for accuracy in per_acc:
        count=0
        for tup in accuracy:
            if tup[1]==tup[2]:
                count+=1
        acc_per_label= count/9
        total_acc.append(acc_per_label)
    return (sum(total_acc)*div)

In [30]:
def advance_calculate_accuracy(x_test, y_test, model, similarity="cosine"):
    predictions=model.predict(x_test)
    predictions /= np.linalg.norm(predictions, axis=1, keepdims=True)
    per_acc=[]
    div=0.8 #total classes that are predicted
    for i in range(0,x_test.shape[0], 10):
        a=predictions[i]
        label_1= np.argmax(y_test[i])
        test_acc=[]
        print("Calculating the {} th sample".format(i//10))
        for j in range(x_test.shape[0]):
            b=predictions[j]
            if np.mean(a) == np.mean(b):
                continue
            label_2 = np.argmax(y_test[j])
            if similarity == "cosine":
                sim = 1- spatial.distance.cosine(a, b)
                test_acc.append((sim, label_1, label_2))
            elif similarity == "euclidean":
                sim = sqrt(sum(pow(x-y,2) for x, y in zip(a, b)))
                test_acc.append((sim, label_1, label_2))
            elif similarity == "manhattan":
                sim = sum(abs(x-y) for x,y in zip(a,b))
                test_acc.append((sim, label_1, label_2))
           
        if similarity=="euclidean" or similarity=="manhattan":
            sorted_test = sorted(test_acc)
            print(sorted_test[:9])
            per_acc.append(sorted_test[:9])
        else:
            sorted_test = sorted(test_acc, reverse=True)
            print(sorted_test[:9])
            per_acc.append(sorted_test[:9])
    
    total_acc=[]
    for accuracy in per_acc:
        count=0
        for tup in accuracy:
            if tup[1]==tup[2]:
                count+=1
        acc_per_label= count/9
        total_acc.append(acc_per_label)
    return (sum(total_acc)*div)

In [29]:
from keras import backend as K
from keras.layers import Layer
from keras import regularizers

import tensorflow as tf


class ArcFace(Layer):
    def __init__(self, n_classes=10, s=30.0, m=0.50, regularizer=None, **kwargs):
        super(ArcFace, self).__init__(**kwargs)
        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.regularizer = regularizers.get(regularizer)

    def build(self, input_shape):
        super(ArcFace, self).build(input_shape[0])
        self.W = self.add_weight(name='W',
                                shape=(input_shape[0][-1], self.n_classes),
                                initializer='glorot_uniform',
                                trainable=True,
                                regularizer=self.regularizer)

    def call(self, inputs):
        x, y = inputs
        c = K.shape(x)[-1]
        # normalize feature
        x = tf.nn.l2_normalize(x, axis=1)
        # normalize weights
        W = tf.nn.l2_normalize(self.W, axis=0)
        # dot product
        logits = x @ W
        # add margin
        # clip logits to prevent zero division when backward
        theta = tf.acos(K.clip(logits, -1.0 + K.epsilon(), 1.0 - K.epsilon()))
        target_logits = tf.cos(theta + self.m)
        # sin = tf.sqrt(1 - logits**2)
        # cos_m = tf.cos(logits)
        # sin_m = tf.sin(logits)
        # target_logits = logits * cos_m - sin * sin_m
        #
        logits = logits * (1 - y) + target_logits * y
        # feature re-scale
        logits *= self.s
        out = tf.nn.softmax(logits)

        return out

    def compute_output_shape(self, input_shape):
        return (None, self.n_classes)


class SphereFace(Layer):
    def __init__(self, n_classes=10, s=30.0, m=1.35, regularizer=None, **kwargs):
        super(SphereFace, self).__init__(**kwargs)
        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.regularizer = regularizers.get(regularizer)

    def build(self, input_shape):
        super(SphereFace, self).build(input_shape[0])
        self.W = self.add_weight(name='W',
                                shape=(input_shape[0][-1], self.n_classes),
                                initializer='glorot_uniform',
                                trainable=True,
                                regularizer=self.regularizer)

    def call(self, inputs):
        x, y = inputs
        c = K.shape(x)[-1]
        # normalize feature
        x = tf.nn.l2_normalize(x, axis=1)
        # normalize weights
        W = tf.nn.l2_normalize(self.W, axis=0)
        # dot product
        logits = x @ W
        # add margin
        # clip logits to prevent zero division when backward
        theta = tf.acos(K.clip(logits, -1.0 + K.epsilon(), 1.0 - K.epsilon()))
        target_logits = tf.cos(self.m * theta)
        #
        logits = logits * (1 - y) + target_logits * y
        # feature re-scale
        logits *= self.s
        out = tf.nn.softmax(logits)

        return out

    def compute_output_shape(self, input_shape):
        return (None, self.n_classes)


class CosFace(Layer):
    def __init__(self, n_classes=10, s=30.0, m=0.35, regularizer=None, **kwargs):
        super(CosFace, self).__init__(**kwargs)
        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.regularizer = regularizers.get(regularizer)

    def build(self, input_shape):
        super(CosFace, self).build(input_shape[0])
        self.W = self.add_weight(name='W',
                                shape=(input_shape[0][-1], self.n_classes),
                                initializer='glorot_uniform',
                                trainable=True,
                                regularizer=self.regularizer)

    def call(self, inputs):
        x, y = inputs
        c = K.shape(x)[-1]
        # normalize feature
        x = tf.nn.l2_normalize(x, axis=1)
        # normalize weights
        W = tf.nn.l2_normalize(self.W, axis=0)
        # dot product
        logits = x @ W
        # add margin
        target_logits = logits - self.m
        #
        logits = logits * (1 - y) + target_logits * y
        # feature re-scale
        logits *= self.s
        out = tf.nn.softmax(logits)

        return out

    def compute_output_shape(self, input_shape):
        return (None, self.n_classes)

In [39]:
def libri_calculate_accuracy(x_test, y_test, model, similarity="cosine"):
    y_true=np.argmax(y_test, axis=1)
    predictions=model.predict(x_test)
    predictions /= np.linalg.norm(predictions, axis=1, keepdims=True)
    total_prediction=0
    per_acc=[]
    match_count=0
    for i in range(0,x_test.shape[0], 10):
        a=predictions[i]
        label_1= np.argmax(y_test[i])
        pick_items = (list(y_true)).count(label_1) -1
        test_acc=[]
        print("Calculating the {} th sample".format(i//10))
        for j in range(x_test.shape[0]):
            b=predictions[j]
            if np.mean(a) == np.mean(b):
                continue
            label_2 = np.argmax(y_test[j])
            if similarity == "cosine":
                sim = 1- spatial.distance.cosine(a, b)
                test_acc.append((sim, label_1, label_2))
            elif similarity == "euclidean":
                sim = sqrt(sum(pow(x-y,2) for x, y in zip(a, b)))
                test_acc.append((sim, label_1, label_2))
            elif similarity == "manhattan":
                sim = sum(abs(x-y) for x,y in zip(a,b))
                test_acc.append((sim, label_1, label_2))
           
        if similarity=="euclidean" or similarity=="manhattan":
            sorted_test = sorted(test_acc)
            print(sorted_test[:pick_items])
            per_acc.append(sorted_test[:pick_items])
        else:
            sorted_test = sorted(test_acc, reverse=True)
            print(sorted_test[:pick_items])
            per_acc.append(sorted_test[:pick_items])
        total_prediction+= (pick_items-1)
    
    for accuracy in per_acc:
        for tup in accuracy:
            if tup[1]==tup[2]:
                #count+=1
                match_count+=1
    return (match_count/total_prediction)*100